# Práctica | Visualización de Datos
## Usando la API de Idealista
En este notebook vamos a llamar a la API de Idealsita para obtener los datos necesarios para el desarrollo del dashboard en Tabelau.

Para ello, primero debemos adquirir nuestro apikey a través de la página web de Idealista.

In [ ]:
#Importamos librerías
import requests
import base64
import pandas as pd
import numpy as np
import json
import urllib.parse

### Ciudades a explorar:

Barcelona: 0-EU-ES-08-13
Sagrada Familia: 0-EU-ES-08-13-001-019-02-005 (5 páginas)

Madrid: 0-EU-ES-28-07
Chamberí: 0-EU-ES-28-07-001-079-07 (31 páginas)

El puerto: 0-EU-ES-11-01-002-027 (11 páginas)

# Código
A continuación tenemos el código a ejecutar para obtener nuestro token, para ello debemos reeemplazar las credenciales que no ha proporcionado Idealista:

In [ ]:
# Tus credenciales
client_id = 'xxxxx'
client_secret = 'xxxxx'

# Codificación base64 de "client_id:client_secret"
credentials = f"{client_id}:{client_secret}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

# URL de autenticación
url = "https://api.idealista.com/oauth/token"

# Cuerpo de la solicitud (formato x-www-form-urlencoded)
data = {
    "grant_type": "client_credentials",
    "scope": "read"
}
encoded_data = urllib.parse.urlencode(data)

# Encabezados HTTP
headers = {
    "Authorization": f"Basic {encoded_credentials}",
    "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"}

# Hacer la solicitud POST
response = requests.post(url, headers=headers, data=encoded_data)

# Mostrar resultados
if response.status_code == 200:
    token_data = response.json()
    print("✅ Access Token:", token_data["access_token"])
    print("Token Type:", token_data["token_type"])
    print("Expires In:", token_data["expires_in"], "seconds")
    print("Scope:", token_data["scope"])
else:
    print("❌ Error:", response.status_code)
    print("Respuesta:", response.text)

Ahora definimos la función para realizar nuestra búsqueda. Cambiaremos los valores a nuestro antojo.

In [ ]:
def define_search_url(page):
    base_url = 'https://api.idealista.com/3.5/es/search'

    params = {
        "operation": "sale",                # o "rent"
        "maxItems": "50",
        "center": "41.3851,2.1734",         # coordenadas de Barcelona, por ejemplo
        "distance": "4000",
        "propertyType": "homes",
        "sort": "desc",
        "numPage": str(page),
        "language": "es"
    }

    # Construir la URL completa con parámetros codificados
    import urllib.parse
    return base_url + "?" + urllib.parse.urlencode(params)

A continución tenemos la función para llamar a la API:

In [ ]:
def search_api(page):
    access_token = token_data["access_token"]
    token = access_token
    url = "https://api.idealista.com/3.5/es/search"

    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {
        "operation": "sale",
        "maxItems": "50",
        "center": "41.3851,2.1734",
        "distance": "4000",
        "propertyType": "homes",
        "sort": "desc",
        "numPage": str(page),
        "language": "es"
    }

    response = requests.post(url, headers=headers, data=data)

    print("Status:", response.status_code)
    print("Response URL:", response.url)
    print("Text:", repr(response.text))

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"❌ Error {response.status_code}: {response.text}")

Obtenemos la página 1 y el total de páginas:

In [ ]:
pagination = 1
results = search_api(1)
results
total_pages = results['totalPages']

Creamos funciones para concatenar las búsquedas realizadas en bucle:

In [ ]:
def results_to_df(results):
    df = pd.DataFrame.from_dict(results['elementList'])
    return df

def concat_df(df, df_tot):
    df_tot = pd.concat([df_tot, df])
    return df_tot

df = results_to_df(results)
df_tot = pd.DataFrame()
df_tot = concat_df(df, df_tot)

Ejecutamos el bucle real donde iremos obteniendo registros de 50 en 50.

In [ ]:
# Bucle de paginación
for i in range(1, total_pages + 1):
    url_page = define_search_url(i)  # Crear URL con número de página
    results = search_api(url_page)   # Obtener resultados
    df = results_to_df(results)      # Convertir a DataFrame
    df_tot = concat_df(df, df_tot)   # Añadir al total
               # Evitar límite de 1 req/segundo

# Guardar CSV
df_tot.to_csv("idealista.csv", index=False)


# Mostrar ruta y dataset
print("CSV guardado")
df_tot.head()

Aquí tendriamos listo nuestro dataset. En caso de que quisieramos obtener más datos repetiriamos el proceso cambiando la URL en la función previamente explicada.

## Limpieza del DataSet

Antes de visualizar nuestro dataset en Tableau, tendremos que hacer un poco de preprocessing. 
Vamos a seleccionar las columnas útiles, borrar duplicados y outliers.

In [ ]:
#Cargamos nuestro dataset generado
df = pd.read_csv("idealista.csv")
df.head()  # Ver primeras filas

In [ ]:
df.shape  # Ver cuántas filas y columnas tiene

In [ ]:
df = df[df["priceInfo"].astype(str).str.contains("€/mes")]

# --- Selección de columnas útiles ---
cols_base = [
    "price", "size", "rooms", "bathrooms", "propertyType",
    "address", "neighborhood", "district", "municipality", "province",
    "latitude", "longitude", "url", "zona"
]

#Outliers
Q1 = df["price"].quantile(0.25)
Q3 = df["price"].quantile(0.75)
IQR = Q3-Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR
df=df[(df["price"]>=lower) & (df["price"]<=upper)]
#Combinar todo
df = df[cols_base]
df_limpio=df.drop_duplicates()

#Guardar CSV limpio
df_limpio.to_csv("alquileres_limpio.csv", index=False)
print("✅ Dataset limpio guardado como 'todo_alquileres_limpio.csv'")

In [ ]:
df_limpio.head()

In [ ]:
df_limpio.shape

Ya tendríamos nuestro dataset perfecto apra ser utilizado en Tableau.